<a href="https://colab.research.google.com/github/Hyun-ho-Lee/Dacon_KorNLI/blob/main/Sentence%20Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers datasets scipy scikit-learn

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 311 kB 45.1 MB/s 
     |████████████████████████████████| 38.1 MB 249 kB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
     |████████████████████████████████| 6.5 MB 26.7 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 134 kB 47.5 MB/s 
     |████████████████████████████████| 1.1 MB 31.2 MB/s 
     |████████████████████████████████| 212 kB 47.6 MB/s 
     |████████████████████████████████| 144 kB 45.6 MB/s 
     |████████████████████████████████| 271 kB 41.5 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.

In [2]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [3]:
model_checkpoint = "klue/roberta-base"
batch_size = 32
task = "nli"

In [4]:
datasets = load_dataset("klue", task)

Downloading:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 3000
    })
})

In [6]:
datasets["train"][0]

{'guid': 'klue-nli-v1_train_00000',
 'hypothesis': '힛걸 진심 최고로 멋지다.',
 'label': 0,
 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다',
 'source': 'NSMC'}

In [7]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,guid,source,premise,hypothesis,label
0,klue-nli-v1_train_18037,policy,자동차 업계에서는 주요 차종 특별할인에 나선다.,일부 차종에는 특별할인이 적용된다.,entailment
1,klue-nli-v1_train_18599,airbnb,전자레인지 냉장고 스탠드 커피포트 다 있고요.,귀여운 미니 냉장고가 있어요.,neutral
2,klue-nli-v1_train_06675,policy,또 여러 정부 부처의 한류관련 정책과 정보가 분산돼 비효율이 있었고 한류의 긍정적 파급효과를 높이는 데 한계가 있었다.,앞으로 한류관련 정책과 정보는 문화체육관광부에서 일체 담당한다.,neutral
3,klue-nli-v1_train_02809,wikinews,그 가족들은 쿠팡의 부적절한 택배 운영을 질책하면서 청와대에 쿠팡이 책임을 다해줄 것을 청원하는 청원서를 올렸다.,청와대 청원서 글에는 수만개의 동의가 올라왔다.,neutral
4,klue-nli-v1_train_01365,wikinews,"강릉과 속초, 동해, 삼척, 고성, 정선 등 30개 노선 시내버스 운행이 단축 운행되고 있다고 노컷뉴스가 보도했다.",강원도에서 30개 노선 시내버스 운행이 단축 운행되고 있다.,entailment
5,klue-nli-v1_train_05824,NSMC,대좌가 아들이 있다는점에서 충격을 받은 영화입니다,대좌는 아들이 있습니다.,entailment
6,klue-nli-v1_train_10204,NSMC,상처 받은 영혼에 촉촉한 비를 뿌려주는 너무 좋은 영화,촉촉한 비가 내리는 영화.,contradiction
7,klue-nli-v1_train_21503,policy,최초제안자가 사업자 선정에서 탈락하는 경우 제3자보다 추가 보상해 최초제안자 보상을 강화한다.,제3자보다 추가 보상해 최초제안자 보상을 강화하는 경우는 최초제한자가 사업자 선정에서 탈락했을 때 이다.,entailment
8,klue-nli-v1_train_20612,airbnb,집 바로 앞에는 트램 정류장이 있어서 편해요.,트램 정류장은 집에서 멀리 떨어져있어요.,contradiction
9,klue-nli-v1_train_14847,wikipedia,원장의 그런 행동으로 친구들은 처음에는 그저 어리둥절하지만 상냥하고 착하며 아름다운 세라의 모습에 친해지기 시작한다.,친구들은 세라를 따돌렸다.,contradiction


In [9]:
metric = load_metric("glue", "qnli")

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

In [10]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
fake_preds, fake_labels

(array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]),
 array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1]))

In [11]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.4375}

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [13]:
tokenizer("힛걸 진심 최고로 멋지다.", "힛걸 진심 최고다 그 어떤 히어로보다 멋지다")

{'input_ids': [0, 3, 7254, 3841, 2200, 11980, 2062, 18, 2, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
sentence1_key, sentence2_key = ("premise", "hypothesis")
print(f"Sentence 1: {datasets['train'][0][sentence1_key]}")
print(f"Sentence 2: {datasets['train'][0][sentence2_key]}")

Sentence 1: 힛걸 진심 최고다 그 어떤 히어로보다 멋지다
Sentence 2: 힛걸 진심 최고로 멋지다.


In [15]:
def preprocess_function(examples):
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        truncation=True,
        return_token_type_ids=False,
    )

In [16]:
preprocess_function(datasets["train"][:5])

{'input_ids': [[0, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2, 3, 7254, 3841, 2200, 11980, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 1525, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 1282, 2955, 3308, 2052, 3944, 11580, 2359, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 5105, 2318, 831, 717, 2886, 2069, 575, 555, 2062, 18, 2], [0, 10522, 2548, 2500, 6328, 2170, 6189, 5916, 4015, 2116, 1039, 2219, 3606, 18, 2, 10522, 2548, 2500, 6328, 27135, 5916, 4015, 1642, 2015, 2259, 4258, 2219, 3606, 18, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1

In [17]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [18]:
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'clas

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
)

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, premise, guid, hypothesis. If source, premise, guid, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24998
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3910


Epoch,Training Loss,Validation Loss,Accuracy
1,0.583200,0.452381,0.830000
2,0.280700,0.454397,0.854667
3,0.176400,0.492729,0.858000
4,0.100800,0.624190,0.855333
5,0.075600,0.714277,0.858000


Saving model checkpoint to test-nli/checkpoint-500
Configuration saved in test-nli/checkpoint-500/config.json
Model weights saved in test-nli/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-nli/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-nli/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, premise, guid, hypothesis. If source, premise, guid, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32
Saving model checkpoint to test-nli/checkpoint-1000
Configuration saved in test-nli/checkpoint-1000/config.json
Model weights saved in test-nli/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-nli/checkpoint-1000/tokenizer_config.json
Special tokens file s

TrainOutput(global_step=3910, training_loss=0.22622300267524426, metrics={'train_runtime': 3094.1751, 'train_samples_per_second': 40.395, 'train_steps_per_second': 1.264, 'total_flos': 4414333976592120.0, 'train_loss': 0.22622300267524426, 'epoch': 5.0})

In [25]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, premise, guid, hypothesis. If source, premise, guid, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.858,
 'eval_loss': 0.7142773270606995,
 'eval_runtime': 23.3462,
 'eval_samples_per_second': 128.501,
 'eval_steps_per_second': 4.026}